In [ ]:
"""
Created on Tue May 28 12:34 2024

This script is to plot the evolution of the different mass fluxes for each ice shelf

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
isf_mask = file_isf['ISF_mask']

In [ ]:
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS.nc')

In [ ]:
mask_basins = xr.open_dataset(inputpath_data + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc')

PREPARE DATA

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #,'GISS-E2-1-H'

In [ ]:
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
SMB_weighted_quantiles = file_viability_info['SMB']

In [ ]:
TMB_quantiles = file_viability_info['MASS_BALANCE'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).quantile([0.33,0.5,0.66],dim=['model','param','m'])


In [ ]:
SMB_quantiles = (-1*file_viability_info['SMB']).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).quantile([0.33,0.5,0.66],dim=['model','param','m'])
BMB_quantiles = file_viability_info['BMB'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).quantile([0.33,0.5,0.66],dim=['model','param','m'])
calv_quantiles = file_viability_info['CALVING'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).quantile([0.33,0.5,0.66],dim=['model','param','m'])
ABUMIP_quantiles = file_viability_info['ABUMIP'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).mean(dim=['model','param','m'])

In [ ]:
file_TS_scen_list = []
for scenario in ['historical','ssp126','ssp245','ssp585']:
    print(scenario)
    
    file_TS_model_list = []
    for mod in tqdm(weight_file.model.values):

        if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
            to2300 = False
        elif mod in ['GISS-E2-1-H']:
            to2300 = True
        elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
            to2300 = True
        elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
            to2300 = False
        elif mod == 'UKESM1-0-LL':
            to2300 = True     
        elif mod == 'CESM2':
            to2300 = False        

        if scenario == 'historical':
            yystart = 1850 #1980 #1850
            yyend = 2014
        elif scenario == 'ssp245':
            yystart = 2015
            yyend = 2100  
        else:
            if to2300:
                yystart = 2015
                yyend = 2300
            else:
                yystart = 2015
                yyend = 2100 

        outputpath_mean = home_path+'/DATA/SUMMER_PAPER/interim/T_S_mean/'+mod+'/'
        
        if scenario == 'ssp126' and mod == 'GFDL-CM4':
            print('beuh')
        else:
            mean_prop = xr.open_dataset(outputpath_mean + 'T_S_TF_isfmean_'+scenario+'.nc')
            file_TS_model_list.append(mean_prop.assign_coords({'model': mod}))
        file_TS1 = xr.concat(file_TS_model_list, dim='model')
    file_TS_scen_list.append(file_TS1.assign_coords({'scenario': scenario}))
file_TS3 = xr.concat(file_TS_scen_list, dim='scenario')

In [ ]:
weighted_prop_2300 = file_TS3.weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).quantile([0.33,0.5,0.66],dim=['model','param','m'])

In [ ]:
scen = 'ssp585'

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)

    
                    
    ax[i].plot(np.arange(1850,2015),SMB_quantiles.sel(quantile=0.5, scenario=scen, time=range(1850,2015), Nisf=kisf), color='orange')
    ax[i].plot(np.arange(1850,2015),BMB_quantiles.sel(quantile=0.5, scenario=scen, time=range(1850,2015), Nisf=kisf), color='c')

    ax[i].plot(np.arange(2015,2300),SMB_quantiles.sel(quantile=0.5, scenario=scen, time=range(2015,2300), Nisf=kisf), color='orange')
    ax[i].plot(np.arange(2015,2300),BMB_quantiles.sel(quantile=0.5, scenario=scen, time=range(2015,2300), Nisf=kisf), color='c')
    
    ax[i].fill_between(np.arange(1850,2015), y1=SMB_quantiles.sel(quantile=0.33, scenario=scen, time=range(1850,2015),Nisf=kisf),
                       y2=SMB_quantiles.sel(quantile=0.66, scenario=scen, time=range(1850,2015),Nisf=kisf), color='orange', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=SMB_quantiles.sel(quantile=0.33, scenario=scen, time=range(2015,2300),Nisf=kisf),
                       y2=SMB_quantiles.sel(quantile=0.66, scenario=scen, time=range(2015,2300),Nisf=kisf), color='orange', alpha=0.25)
    ax[i].fill_between(np.arange(1850,2015), y1=BMB_quantiles.sel(quantile=0.33, scenario=scen, time=range(1850,2015),Nisf=kisf),
                       y2=BMB_quantiles.sel(quantile=0.66, scenario=scen, time=range(1850,2015),Nisf=kisf), color='c', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=BMB_quantiles.sel(quantile=0.33, scenario=scen, time=range(2015,2300),Nisf=kisf),
                       y2=BMB_quantiles.sel(quantile=0.66, scenario=scen, time=range(2015,2300),Nisf=kisf), color='c', alpha=0.25)
    
    ax[i].axhline(y=calv_quantiles.sel(quantile=0.5, scenario=scen, time=range(2015,2300), Nisf=kisf).max(), color='olive')
    ax[i].axhline(y=ABUMIP_quantiles.sel(scenario=scen, time=range(2015,2300), Nisf=kisf).max(), color='red')

    
    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    ax[i].set_xticks([1900,2000,2100,2200,2300])
    if i < 56:
        ax[i].tick_params(labelbottom=False)
    else:
        ax[i].set_xticklabels([1900, 2000, 2100, 2200, 2300], rotation=45)
    
            
    i = i+1
    
sns.despine()
plt.tight_layout()
plt.savefig(plot_path + 'mass_fluxes_evolution_'+scen+'_withoutGISS.png', dpi=250)

In [ ]:

f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)

    
                    
    ax[i].plot(np.arange(1850,2015),BMB_quantiles.sel(quantile=0.5, scenario='ssp126', time=range(1850,2015), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(1850,2015),BMB_quantiles.sel(quantile=0.5, scenario='ssp585', time=range(1850,2015), Nisf=kisf), color='darkmagenta')

    ax[i].plot(np.arange(2015,2300),BMB_quantiles.sel(quantile=0.5, scenario='ssp126', time=range(2015,2300), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(2015,2300),BMB_quantiles.sel(quantile=0.5, scenario='ssp585', time=range(2015,2300), Nisf=kisf), color='darkmagenta')
    
    ax[i].fill_between(np.arange(1850,2015), y1=BMB_quantiles.sel(quantile=0.33, scenario='ssp126', time=range(1850,2015),Nisf=kisf),
                       y2=BMB_quantiles.sel(quantile=0.66, scenario='ssp126', time=range(1850,2015),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=BMB_quantiles.sel(quantile=0.33, scenario='ssp126', time=range(2015,2300),Nisf=kisf),
                       y2=BMB_quantiles.sel(quantile=0.66, scenario='ssp126', time=range(2015,2300),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(1850,2015), y1=BMB_quantiles.sel(quantile=0.33, scenario='ssp585', time=range(1850,2015),Nisf=kisf),
                       y2=BMB_quantiles.sel(quantile=0.66, scenario='ssp585', time=range(1850,2015),Nisf=kisf), color='darkmagenta', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=BMB_quantiles.sel(quantile=0.33, scenario='ssp585', time=range(2015,2300),Nisf=kisf),
                       y2=BMB_quantiles.sel(quantile=0.66, scenario='ssp585', time=range(2015,2300),Nisf=kisf), color='darkmagenta', alpha=0.25)
    

    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    #ax[i].axvline(x=2100)
    ax[i].set_xticks([1900,2000,2100,2200,2300])
    if i < 56:
        ax[i].tick_params(labelbottom=False)
    else:
        ax[i].set_xticklabels([1900, 2000, 2100, 2200, 2300], rotation=45)
            
    i = i+1
    
sns.despine()
plt.tight_layout()
plt.savefig(plot_path + 'BMB_fluxes_evolution_'+scen+'_withoutGISS.png', dpi=250)

In [ ]:

f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)

    
                    
    ax[i].plot(np.arange(1850,2015),TMB_quantiles.sel(quantile=0.5, scenario='ssp126', time=range(1850,2015), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(1850,2015),TMB_quantiles.sel(quantile=0.5, scenario='ssp585', time=range(1850,2015), Nisf=kisf), color='darkmagenta')

    ax[i].plot(np.arange(2015,2300),TMB_quantiles.sel(quantile=0.5, scenario='ssp126', time=range(2015,2300), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(2015,2300),TMB_quantiles.sel(quantile=0.5, scenario='ssp585', time=range(2015,2300), Nisf=kisf), color='darkmagenta')
    
    ax[i].fill_between(np.arange(1850,2015), y1=TMB_quantiles.sel(quantile=0.33, scenario='ssp126', time=range(1850,2015),Nisf=kisf),
                       y2=TMB_quantiles.sel(quantile=0.66, scenario='ssp126', time=range(1850,2015),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=TMB_quantiles.sel(quantile=0.33, scenario='ssp126', time=range(2015,2300),Nisf=kisf),
                       y2=TMB_quantiles.sel(quantile=0.66, scenario='ssp126', time=range(2015,2300),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(1850,2015), y1=TMB_quantiles.sel(quantile=0.33, scenario='ssp585', time=range(1850,2015),Nisf=kisf),
                       y2=TMB_quantiles.sel(quantile=0.66, scenario='ssp585', time=range(1850,2015),Nisf=kisf), color='darkmagenta', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=TMB_quantiles.sel(quantile=0.33, scenario='ssp585', time=range(2015,2300),Nisf=kisf),
                       y2=TMB_quantiles.sel(quantile=0.66, scenario='ssp585', time=range(2015,2300),Nisf=kisf), color='darkmagenta', alpha=0.25)
    

    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    #ax[i].axvline(x=2100)
    ax[i].set_xticks([1900,2000,2100,2200,2300])
    if i < 56:
        ax[i].tick_params(labelbottom=False)
    else:
        ax[i].set_xticklabels([1900, 2000, 2100, 2200, 2300], rotation=45)
            
    i = i+1
    
sns.despine()
plt.tight_layout()
plt.savefig(plot_path + 'TMB_fluxes_evolution_bothscen_withoutGISS.png', dpi=250)

In [ ]:

f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)

    
                    
    ax[i].plot(np.arange(1850,2015),SMB_quantiles.sel(quantile=0.5, scenario='ssp126', time=range(1850,2015), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(1850,2015),SMB_quantiles.sel(quantile=0.5, scenario='ssp585', time=range(1850,2015), Nisf=kisf), color='darkmagenta')

    ax[i].plot(np.arange(2015,2300),SMB_quantiles.sel(quantile=0.5, scenario='ssp126', time=range(2015,2300), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(2015,2300),SMB_quantiles.sel(quantile=0.5, scenario='ssp585', time=range(2015,2300), Nisf=kisf), color='darkmagenta')
    
    ax[i].fill_between(np.arange(1850,2015), y1=SMB_quantiles.sel(quantile=0.33, scenario='ssp126', time=range(1850,2015),Nisf=kisf),
                       y2=SMB_quantiles.sel(quantile=0.66, scenario='ssp126', time=range(1850,2015),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=SMB_quantiles.sel(quantile=0.33, scenario='ssp126', time=range(2015,2300),Nisf=kisf),
                       y2=SMB_quantiles.sel(quantile=0.66, scenario='ssp126', time=range(2015,2300),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(1850,2015), y1=SMB_quantiles.sel(quantile=0.33, scenario='ssp585', time=range(1850,2015),Nisf=kisf),
                       y2=SMB_quantiles.sel(quantile=0.66, scenario='ssp585', time=range(1850,2015),Nisf=kisf), color='darkmagenta', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=SMB_quantiles.sel(quantile=0.33, scenario='ssp585', time=range(2015,2300),Nisf=kisf),
                       y2=SMB_quantiles.sel(quantile=0.66, scenario='ssp585', time=range(2015,2300),Nisf=kisf), color='darkmagenta', alpha=0.25)
    

    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    #ax[i].axvline(x=2100)
    ax[i].set_xticks([1900,2000,2100,2200,2300])
    if i < 56:
        ax[i].tick_params(labelbottom=False)
    else:
        ax[i].set_xticklabels([1900, 2000, 2100, 2200, 2300], rotation=45)
            
    i = i+1
    
sns.despine()
plt.tight_layout()
plt.savefig(plot_path + 'SMB_fluxes_evolution_'+scen+'_withoutGISS.png', dpi=250)

In [ ]:
var = 'TF_mean'

f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)

    
                    
    ax[i].plot(np.arange(1850,2015),weighted_prop_2300[var].sel(quantile=0.5, scenario='ssp126', time=range(1850,2015), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(1850,2015),weighted_prop_2300[var].sel(quantile=0.5, scenario='ssp585', time=range(1850,2015), Nisf=kisf), color='darkmagenta')

    ax[i].plot(np.arange(2015,2300),weighted_prop_2300[var].sel(quantile=0.5, scenario='ssp126', time=range(2015,2300), Nisf=kisf), color='pink')
    ax[i].plot(np.arange(2015,2300),weighted_prop_2300[var].sel(quantile=0.5, scenario='ssp585', time=range(2015,2300), Nisf=kisf), color='darkmagenta')
    
    ax[i].fill_between(np.arange(1850,2015), y1=weighted_prop_2300[var].sel(quantile=0.33, scenario='ssp126', time=range(1850,2015),Nisf=kisf),
                       y2=weighted_prop_2300[var].sel(quantile=0.66, scenario='ssp126', time=range(1850,2015),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=weighted_prop_2300[var].sel(quantile=0.33, scenario='ssp126', time=range(2015,2300),Nisf=kisf),
                       y2=weighted_prop_2300[var].sel(quantile=0.66, scenario='ssp126', time=range(2015,2300),Nisf=kisf), color='pink', alpha=0.25)
    ax[i].fill_between(np.arange(1850,2015), y1=weighted_prop_2300[var].sel(quantile=0.33, scenario='ssp585', time=range(1850,2015),Nisf=kisf),
                       y2=weighted_prop_2300[var].sel(quantile=0.66, scenario='ssp585', time=range(1850,2015),Nisf=kisf), color='darkmagenta', alpha=0.25)
    ax[i].fill_between(np.arange(2015,2300), y1=weighted_prop_2300[var].sel(quantile=0.33, scenario='ssp585', time=range(2015,2300),Nisf=kisf),
                       y2=weighted_prop_2300[var].sel(quantile=0.66, scenario='ssp585', time=range(2015,2300),Nisf=kisf), color='darkmagenta', alpha=0.25)
    

    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    #ax[i].axvline(x=2100)
    ax[i].set_xticks([1900,2000,2100,2200,2300])
    ax[i].set_ylim(0,6)

    if i < 56:
        ax[i].tick_params(labelbottom=False)
    else:
        ax[i].set_xticklabels([1900, 2000, 2100, 2200, 2300], rotation=45)
            
    i = i+1
    
sns.despine()
plt.tight_layout()


In [ ]:
### relationship seems to be isf specific
plt.figure()
plt.scatter(weighted_prop_2300[var].sel(quantile=0.5, time=range(2015,2300),scenario='ssp585'),BMB_quantiles.sel(quantile=0.5, time=range(2015,2300),scenario='ssp585'),alpha=0.05,c='darkmagenta')
plt.scatter(weighted_prop_2300[var].sel(quantile=0.5, time=range(2015,2300),scenario='ssp126'),BMB_quantiles.sel(quantile=0.5, time=range(2015,2300),scenario='ssp126'),alpha=0.05,c='pink')


In [ ]:
f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)
    
    ax[i].scatter(weighted_prop_2300[var].sel(quantile=0.5, time=range(2015,2300),scenario='ssp585',Nisf=kisf),TMB_quantiles.sel(quantile=0.5, time=range(2015,2300),scenario='ssp585',Nisf=kisf),alpha=0.05,c='darkmagenta')
    ax[i].scatter(weighted_prop_2300[var].sel(quantile=0.5, time=range(2015,2300),scenario='ssp126',Nisf=kisf),TMB_quantiles.sel(quantile=0.5, time=range(2015,2300),scenario='ssp126',Nisf=kisf),alpha=0.05,c='pink')
    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)

    i = i+1
    
sns.despine()
plt.tight_layout()

In [ ]:
f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)
    
    ax[i].scatter(weighted_prop_2300[var].sel(quantile=0.5, time=range(2015,2300),scenario='ssp585',Nisf=kisf),BMB_quantiles.sel(quantile=0.5, time=range(2015,2300),scenario='ssp585',Nisf=kisf),alpha=0.05,c='darkmagenta')
    ax[i].scatter(weighted_prop_2300[var].sel(quantile=0.5, time=range(2015,2300),scenario='ssp126',Nisf=kisf),BMB_quantiles.sel(quantile=0.5, time=range(2015,2300),scenario='ssp126',Nisf=kisf),alpha=0.05,c='pink')
    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)

    i = i+1
    
sns.despine()
plt.tight_layout()

In [ ]:
TMB_weighted_mean = file_viability_info['MASS_BALANCE'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).mean(['param','m'])
TMB_weighted_quantiles = file_viability_info['MASS_BALANCE'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300)).quantile([0.33,0.5,0.66],dim=['param','m'])


In [ ]:
for mparam in bay_weights_2300.param:
    print(mparam.values,(bay_weights_2300 * sens_weights.sel(model=model_2300)).sum(['model','m','Nisf']).sel(param=mparam).values)

In [ ]:
file_TS3[var].sel(time=range(2015,2300),scenario='ssp585',Nisf=kisf)

In [ ]:
var = 'T_mean'
f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1) #, sharex=ax[0]
    else:
        ax[i] = f.add_subplot(8,8,i+1)
    
    ax[i].scatter(file_TS3[var].sel(time=range(2015,2300),scenario='ssp585',Nisf=kisf,model=model_2300),-1*TMB_weighted_mean.sel(time=range(2015,2300),scenario='ssp585',Nisf=kisf),alpha=0.05,c='darkmagenta',s=5)
    ax[i].scatter(file_TS3[var].sel(time=range(2015,2300),scenario='ssp126',Nisf=kisf,model=model_2300),-1*TMB_weighted_mean.sel(time=range(2015,2300),scenario='ssp126',Nisf=kisf),alpha=0.05,c='pink',s=5)
    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    ax[i].set_xlim(-2.5,5)
    ax[i].axhline(0)

    i = i+1
    
sns.despine()
plt.tight_layout()
plt.savefig(plot_path + 'scatter_'+var+'_TMB_weighted_mean.png', dpi=250)

In [ ]:
var = 'T_mean'
f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1) #, sharex=ax[0]
    else:
        ax[i] = f.add_subplot(8,8,i+1)
    
    ax[i].scatter(file_TS3[var].sel(time=range(2015,2300),scenario='ssp585',Nisf=kisf,model=model_2300),TMB_weighted_quantiles.sel(quantile=0.66,time=range(2015,2300),scenario='ssp585',Nisf=kisf),alpha=0.05,c='darkmagenta',s=5)
    ax[i].scatter(file_TS3[var].sel(time=range(2015,2300),scenario='ssp126',Nisf=kisf,model=model_2300),TMB_weighted_quantiles.sel(quantile=0.66,time=range(2015,2300),scenario='ssp126',Nisf=kisf),alpha=0.05,c='pink',s=5)
    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
    ax[i].set_xlim(-2.5,5)
    ax[i].axhline(0)

    i = i+1
    
sns.despine()
plt.tight_layout()
plt.savefig(plot_path + 'scatter_'+var+'_TMB_weighted_quant066.png', dpi=250)

In [ ]:
file_TS3_br, _ = xr.broadcast(file_TS3.sel(model=model_2300,scenario=['ssp126','ssp585'],time=range(2015,2300)),file_viability_info['MASS_BALANCE'].sel(model=model_2300,scenario=['ssp126','ssp585'],time=range(2015,2300)))

In [ ]:
var = 'T_mean'
f = plt.figure()
f.set_size_inches(8.25*1.75, 8.25*1.75)

ax={}

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    if i > 0:
        ax[i] = f.add_subplot(8,8,i+1, sharex=ax[0])
    else:
        ax[i] = f.add_subplot(8,8,i+1)
    
    ax[i].scatter(file_TS3_br[var].sel(time=range(2015,2300),scenario='ssp585',Nisf=kisf,model=model_2300),file_viability_info['MASS_BALANCE'].sel(time=range(2015,2300),scenario='ssp585',Nisf=kisf,model=model_2300),
                  alpha=(bay_weights_2300 * sens_weights.sel(model=model_2300)),c='darkmagenta',s=5)
    ax[i].scatter(file_TS3_br[var].sel(time=range(2015,2300),scenario='ssp126',Nisf=kisf,model=model_2300),file_viability_info['MASS_BALANCE'].sel(time=range(2015,2300),scenario='ssp126',Nisf=kisf,model=model_2300),
                  alpha=(bay_weights_2300 * sens_weights.sel(model=model_2300)),c='pink',s=5)
    ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)

    i = i+1
    
sns.despine()
plt.tight_layout()

In [ ]:
scen = 'ssp126'

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

i = 0


for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot)):
    
    ax[i] = f.add_subplot(8,8,i+1)

    for mmod in model_2300:


        #ax[i].plot(np.arange(1850,2015),file_viability_info['BMB'].sel(model=mmod,param='NNsmall',m=1, scenario=scen, time=range(1850,2015), Nisf=kisf),label=mmod)

        ax[i].plot(np.arange(2015,2300),file_viability_info['BMB'].sel(model=mmod,param='NNsmall',m=1, scenario=scen, time=range(2015,2300), Nisf=kisf),label=mmod)

        ax[i].set_title(file_isf['isf_name'].sel(Nisf=kisf).values)
        ax[i].axvline(x=2100)

    i = i+1

sns.despine()
plt.tight_layout()
plt.legend()
#plt.savefig(plot_path + 'mass_fluxes_evolution_'+scen+'.pdf')

In [ ]:
plt.close('all')

In [ ]:
BMB_ssp585_median = 
(BMB_quantiles.sel(quantile=0.5) - BMB_quantiles.sel(quantile=0.5).shift(time=20)).max(['time'])